## GCS からのダウンロードテスト

In [9]:
from google.cloud import storage
import os
import dotenv

dotenv.load_dotenv(".env")

True

In [7]:
client = storage.Client(os.environ.get("PROJECT_ID"))

/home/yudai/.pyenv/versions/3.7.3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [11]:
username = os.environ.get("USER_ID")

bucket = client.bucket(os.environ.get("BUCKET"))
blob = bucket.blob(f"{username}/AHE/Chiral Hall Effect in Noncollinear Magnets from a Cyclic Cohomology Approach.pdf")

In [12]:
blob.download_to_filename("test.pdf")

## PDF の解析テスト

### PyPDF2

In [13]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 7.8 MB/s  eta 0:00:01
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61084 sha256=816ca41acd3c623c784f3f5b3cdf5d300d97e2b048e7dad97827957d269073fb
  Stored in directory: /home/yudai/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [14]:
import PyPDF2

In [18]:
with open("test.pdf", "rb") as f:
    reader = PyPDF2.PdfFileReader(f)
    page = reader.getPage(0)
    text = page.extractText()

In [20]:
text

' ChiralHallEffectinNoncollinearMagnetsfromaCyclicCohomologyApproach\nFabianR.Lux,\n1,2,*FrankFreimuth,\n1StefanBlügel,\n1andYuriyMokrousov\n1,31PeterGrünbergInstitutandInstituteforAdvancedSimulation,ForschungszentrumJülichandJARA,52425Jülich,Germany\n2DepartmentofPhysics,RWTHAachenUniversity,52056Aachen,Germany\n3InstituteofPhysics,JohannesGutenbergUniversityMainz,55099Mainz,Germany\n(Received21October2019;revisedmanuscriptreceived22December2019;accepted6February2020;published4March2020)\nWedemonstratetheemergenceofananomalousHalleffectinchiralmagnetictextureswhichisneither\nproportionaltothenetmagnetizationnortothewell-knownemergentmagneticfieldthatisresponsiblefor\n\nthetopologicalHalleffect.Instead,itappearsalreadyatlinearorderinthegradientsofthemagnetization\ntextureandexistsforone-dimensionalmagnetictexturessuchasdomainwallsandspinspirals.Itreceivesa\nnaturalinterpretationinthelanguageofAlainConnes\n™noncommutativegeometry.Weshowthatthischiral\nHalleffectresemblesthefamiliartopol

単語間のスペースが消失している。ボツ

### pdfminer.six

In [21]:
!pip install pdfminer.six

In [22]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [32]:
output_string = StringIO()
with open('test.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        text = interpreter.process_page(page)
        break
text = output_string.getvalue()
print(text)

PHYSICAL REVIEW LETTERS 124, 096602 (2020)

Chiral Hall Effect in Noncollinear Magnets from a Cyclic Cohomology Approach

Fabian R. Lux ,1,2,* Frank Freimuth ,1 Stefan Blügel

,1 and Yuriy Mokrousov 1,3

1Peter Grünberg Institut and Institute for Advanced Simulation, Forschungszentrum Jülich and JARA, 52425 Jülich, Germany

2Department of Physics, RWTH Aachen University, 52056 Aachen, Germany

3Institute of Physics, Johannes Gutenberg University Mainz, 55099 Mainz, Germany

 

(Received 21 October 2019; revised manuscript received 22 December 2019; accepted 6 February 2020; published 4 March 2020)

We demonstrate the emergence of an anomalous Hall effect in chiral magnetic textures which is neither
proportional to the net magnetization nor to the well-known emergent magnetic field that is responsible for
the topological Hall effect. Instead, it appears already at linear order in the gradients of the magnetization
texture and exists for one-dimensional magnetic textures such as domain w

改行等も含めて元の PDF の構造を残している。これを元に解析を進めていく。

In [33]:
import re

In [134]:
text2 = re.sub(r"\n+|,|\s+", " ", text.lower())
text3 = re.sub(r"\[\d+((–|,|\s)\d+)?]", "", text2)
text4 = re.sub(r"\.|:|'|’|/|;|\*", "", text3)
text5 = re.sub(r"\-\s", "", text4)
text6 = re.sub(r"\s+", " ", text5)
text7 = re.sub(r"\(|\)", "", text6)

In [136]:
text7

'physical review letters 124 096602 2020 chiral hall effect in noncollinear magnets from a cyclic cohomology approach fabian r lux 1 2 frank freimuth 1 stefan blügel 1 and yuriy mokrousov 1 3 1peter grünberg institut and institute for advanced simulation forschungszentrum jülich and jara 52425 jülich germany 2department of physics rwth aachen university 52056 aachen germany 3institute of physics johannes gutenberg university mainz 55099 mainz germany received 21 october 2019 revised manuscript received 22 december 2019 accepted 6 february 2020 published 4 march 2020 we demonstrate the emergence of an anomalous hall effect in chiral magnetic textures which is neither proportional to the net magnetization nor to the well-known emergent magnetic field that is responsible for the topological hall effect instead it appears already at linear order in the gradients of the magnetization texture and exists for one-dimensional magnetic textures such as domain walls and spin spirals it receives a

In [137]:
len(text)

5219

In [139]:
len(text7)

4982

In [131]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 9.6 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449902 sha256=3b01529ebbf6ea5d6c637610e58ac510b859ebc77de5a03eb3a1f2cef0ff9569
  Stored in directory: /home/yudai/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built nltk


In [146]:
from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/yudai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [147]:
text_list = text7.split(" ")

In [148]:
len(text_list)

755

In [153]:
text_list2 = [word for word in text_list if word not in stopwords.words("english") and len(word) > 1]

In [154]:
len(text_list2)

449

In [155]:
" ".join(text_list2)

'physical review letters 124 096602 2020 chiral hall effect noncollinear magnets cyclic cohomology approach fabian lux frank freimuth stefan blügel yuriy mokrousov 1peter grünberg institut institute advanced simulation forschungszentrum jülich jara 52425 jülich germany 2department physics rwth aachen university 52056 aachen germany 3institute physics johannes gutenberg university mainz 55099 mainz germany received 21 october 2019 revised manuscript received 22 december 2019 accepted february 2020 published march 2020 demonstrate emergence anomalous hall effect chiral magnetic textures neither proportional net magnetization well-known emergent magnetic field responsible topological hall effect instead appears already linear order gradients magnetization texture exists one-dimensional magnetic textures domain walls spin spirals receives natural interpretation language alain connes noncommutative geometry show chiral hall effect resembles familiar topological hall effect essential propert